In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import pandas as pd
from dotenv import load_dotenv
from transformers import pipeline

load_dotenv("../.env")
hf_token = os.getenv("HF_TOKEN")

In [ ]:
eng_data = pd.read_excel("eng_data.xlsx")
model = "tiiuae/falcon-rw-1b"
pipe = pipeline("text-generation", model=model, trust_remote_code=True, token=hf_token)

eng_responses = []
for _, row in eng_data.iterrows():
    text = row["text"]
    prompt = f"<INST>Given the following text, please generate a prompt that it could be a response for:\n\"{text}\"</INST>"

    result = pipe(prompt, max_new_tokens=50)
    generated = result.split("</INST>")[-1].strip()

    eng_responses.append(generated)
eng_data["prompt"] = eng_responses
eng_data.to_excel("eng_data_pseudoprompt.xlsx", index=False)